<a href="https://colab.research.google.com/github/yassineselmi/langchain-workshop/blob/main/lab/02_langchain_conversational_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversational Memory

Conversational memory is how chatbots can respond to our queries in a chat-like manner. It enables a coherent conversation, and without it, every query would be treated as an entirely independent input without considering past interactions.

The memory allows a _"agent"_ to remember previous interactions with the user. By default, agents are *stateless* — meaning each incoming query is processed independently of other interactions. The only thing that exists for a stateless agent is the current input, nothing else.

There are many applications where remembering previous interactions is very important, such as chatbots. Conversational memory allows us to do that.

In this notebook we'll explore this form of memory in the context of the LangChain library.

We'll start by importing all of the libraries that we'll be using in this example.

In [ ]:
!pip install -qU langchain langchain_community langchain-openai tiktoken

In [2]:
import inspect

from getpass import getpass
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key when prompted.

In [ ]:
OPENAI_API_KEY = getpass()

In [4]:
llm = ChatOpenAI(
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo'
)

Later we will make use of a `count_tokens` utility function. This will allow us to count the number of tokens we are using for each call. We define it as so:

In [14]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.invoke(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

Now let's dive into **Conversational Memory**.

## What is memory?

**Definition**: Memory is an agent's capacity of remembering previous interactions with the user (think chatbots)

The official definition of memory is the following:


> By default, Chains and Agents are stateless, meaning that they treat each incoming query independently. In some applications (chatbots being a GREAT example) it is highly important to remember previous interactions, both at a short term but also at a long term level. The concept of “Memory” exists to do exactly that.


As we will see, although this sounds really straightforward there are several different ways to implement this memory capability.

Before we delve into the different memory modules that the library offers, we will introduce the chain we will be using for these examples: the `ConversationChain`.

As always, when understanding a chain it is interesting to peek into its prompt first and then take a look at its `._call` method. As we saw in the chapter on chains, we can check out the prompt by accessing the `template` within the `prompt` attribute.

In [15]:
conversation = ConversationChain(
    llm=llm,
)

In [ ]:
print(conversation.prompt.template)

Interesting! So this chain's prompt is telling it to chat with the user and try to give truthful answers. If we look closely, there is a new component in the prompt that we didn't see when we were tinkering with the `LLMChain`: _history_. This is where our memory will come into play.

So basically this chain combines an input from the user with the conversation history to generate a meaningful (and hopefully truthful) response.

Now that we've understood the basics of the chain we'll be using, we can get into memory. Let's dive in!

## Memory types

In this section we will review several memory types and analyze the pros and cons of each one, so you can choose the best one for your use case.

### Memory type #1: ConversationBufferMemory

The `ConversationBufferMemory` does just what its name suggests: it keeps a buffer of the previous conversation excerpts as part of the context in the prompt.

**Key feature:** _the conversation buffer memory keeps the previous pieces of conversation completely unmodified, in their raw form._

In [17]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

We pass a user prompt the the `ConversationBufferMemory` like so:

In [ ]:
conversation_buf.invoke("Good morning AI!")

This one call used a total of `85` tokens, but we can't see that from the above. If we'd like to count the number of tokens being used we just pass our conversation chain object and the message we'd like to input via the `count_tokens` function we defined earlier:

In [ ]:
count_tokens(
    conversation_buf,
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

In [ ]:
count_tokens(
    conversation_buf,
    "I just want to analyze the different possibilities. What can you think of?"
)

In [ ]:
count_tokens(
    conversation_buf,
    "Which data source types could be used to give context to the model?"
)

In [ ]:
count_tokens(
    conversation_buf,
    "What is my aim again?"
)

Our LLM with `ConversationBufferMemory` can clearly remember earlier interactions in the conversation. Let's take a closer look to how the LLM is saving our previous conversation. We can do this by accessing the `.buffer` attribute for the `.memory` in our chain.

In [ ]:
print(conversation_buf.memory.buffer)

Nice! So every piece of our conversation has been explicitly recorded and sent to the LLM in the prompt.

### Memory type #2: ConversationSummaryMemory

The problem with the `ConversationBufferMemory` is that as the conversation progresses, the token count of our context history adds up. This is problematic because we might max out our LLM with a prompt that is too large to be processed.

Enter `ConversationSummaryMemory`.

Again, we can infer from the name what is going on.. we will keep a summary of our previous conversation snippets as our history. How will we summarize these? LLM to the rescue.

**Key feature:** _the conversation summary memory keeps the previous pieces of conversation in a summarized form, where the summarization is performed by an LLM._

In this case we need to send the llm to our memory constructor to power its summarization ability.

In [24]:
conversation_sum = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

When we have an llm, we always have a prompt ;) Let's see what's going on inside our conversation summary memory:

In [25]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


Cool! So each new interaction is summarized and appended to a running summary as the memory of our chain. Let's see how this works in practice!

In [26]:
# without count_tokens we'd call `conversation_sum("Good morning AI!")`
# but let's keep track of our tokens:
count_tokens(
    conversation_sum,
    "Good morning AI!"
)

Spent a total of 249 tokens


{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning! How can I assist you today?'}

In [27]:
count_tokens(
    conversation_sum,
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 380 tokens


{'input': 'My interest here is to explore the potential of integrating Large Language Models with external knowledge',
 'history': 'The human greets the AI and the AI responds politely, asking how it can assist the human.',
 'response': 'Hello! That sounds like an interesting topic. How can I assist you in exploring the potential of integrating Large Language Models with external knowledge?'}

In [28]:
count_tokens(
    conversation_sum,
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 814 tokens


{'input': 'I just want to analyze the different possibilities. What can you think of?',
 'history': 'The human greets the AI and the AI responds politely, asking how it can assist the human. The human expresses their interest in exploring the potential of integrating Large Language Models with external knowledge. The AI responds enthusiastically, offering its assistance in exploring this topic.',
 'response': "Hello! I'm here to assist you. When it comes to integrating Large Language Models with external knowledge, there are several possibilities to consider. One approach is to use pre-trained models like GPT-3 and fine-tune them on specific tasks or domains using external knowledge. This can help improve the model's performance and make it more specialized in certain areas.\n\nAnother possibility is to leverage existing knowledge graphs or databases to enhance the model's understanding and generate more accurate responses. By connecting the model to external sources of information, it

In [29]:
count_tokens(
    conversation_sum,
    "Which data source types could be used to give context to the model?"
)

Spent a total of 1027 tokens


{'input': 'Which data source types could be used to give context to the model?',
 'history': 'The human greets the AI and the AI responds politely, offering its assistance. The human expresses interest in exploring the potential of integrating Large Language Models with external knowledge. The AI enthusiastically provides several possibilities, including fine-tuning pre-trained models, leveraging existing knowledge graphs or databases, and using knowledge distillation to create more efficient models. The AI offers to provide more information or answer any other questions the human may have.',
 'response': "There are several data source types that can be used to give context to a model. One option is to leverage existing knowledge graphs or databases, such as Wikidata or DBpedia, which contain structured information about various topics. These knowledge graphs can provide valuable context and facts that can be incorporated into the model's understanding.\n\nAnother option is to use web 

In [30]:
count_tokens(
    conversation_sum,
    "What is my aim again?"
)

Spent a total of 586 tokens


{'input': 'What is my aim again?',
 'history': "The human greets the AI and the AI responds politely, offering its assistance. The human expresses interest in exploring the potential of integrating Large Language Models with external knowledge. The AI enthusiastically provides several possibilities, including fine-tuning pre-trained models, leveraging existing knowledge graphs or databases, using web scraping techniques, and utilizing domain-specific datasets. The AI mentions that these data source types can provide valuable context and facts to enhance the model's understanding. The AI offers to provide more information or answer any other questions the human may have.",
 'response': 'Your aim is to explore the potential of integrating Large Language Models with external knowledge.'}

In [31]:
print(conversation_sum.memory.buffer)

The human greets the AI and the AI responds politely, offering its assistance. The human expresses interest in exploring the potential of integrating Large Language Models with external knowledge. The AI enthusiastically provides several possibilities, including fine-tuning pre-trained models, leveraging existing knowledge graphs or databases, using web scraping techniques, and utilizing domain-specific datasets. The AI mentions that these data source types can provide valuable context and facts to enhance the model's understanding. The AI offers to provide more information or answer any other questions the human may have. The human asks what their aim is, and the AI responds that their aim is to explore the potential of integrating Large Language Models with external knowledge.


You might be wondering.. if the aggregate token count is greater in each call here than in the buffer example, why should we use this type of memory? Well, if we check out buffer we will realize that although we are using more tokens in each instance of our conversation, our final history is shorter. This will enable us to have many more interactions before we reach our prompt's max length, making our chatbot more robust to longer conversations.

We can count the number of tokens being used (without making a call to OpenAI) using the `tiktoken` tokenizer like so:

In [34]:
# initialize tokenizer
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

# show number of tokens for the memory used by each memory type
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}'
)

Buffer memory conversation length: 1009
Summary memory conversation length: 133


_Practical Note: the `text-davinci-003` and `gpt-3.5-turbo` models [have](https://platform.openai.com/docs/api-reference/completions/create#completions/create-max_tokens) a large max tokens count of 4096 tokens between prompt and answer._

### Memory type #3: ConversationBufferWindowMemory

Another great option for these cases is the `ConversationBufferWindowMemory` where we will be keeping a few of the last interactions in our memory but we will intentionally drop the oldest ones - short-term memory if you'd like. Here the aggregate token count **and** the per-call token count will drop noticeably. We will control this window with the `k` parameter.

**Key feature:** _the conversation buffer window memory keeps the latest pieces of the conversation in raw form_

In [35]:
conversation_bufw = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1)
)

In [36]:
count_tokens(
    conversation_bufw,
    "Good morning AI!"
)

Spent a total of 77 tokens


{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning! How can I assist you today?'}

In [37]:
count_tokens(
    conversation_bufw,
    "My interest here is to explore the potential of integrating Large Language Models with external knowledge"
)

Spent a total of 374 tokens


{'input': 'My interest here is to explore the potential of integrating Large Language Models with external knowledge',
 'history': 'Human: Good morning AI!\nAI: Good morning! How can I assist you today?',
 'response': "That's a fascinating topic! Integrating Large Language Models with external knowledge can greatly enhance their capabilities and make them more useful in various applications. By incorporating external knowledge sources, such as databases, knowledge graphs, or even the internet, language models can access a vast amount of information to provide more accurate and contextually relevant responses.\n\nThere are several approaches to integrating external knowledge with language models. One common method is to pretrain the language model on a large corpus of text that includes both general knowledge and domain-specific information. This helps the model learn to generate responses that are informed by external knowledge.\n\nAnother approach is to use knowledge graphs, which rep

In [38]:
count_tokens(
    conversation_bufw,
    "I just want to analyze the different possibilities. What can you think of?"
)

Spent a total of 694 tokens


{'input': 'I just want to analyze the different possibilities. What can you think of?',
 'history': "Human: My interest here is to explore the potential of integrating Large Language Models with external knowledge\nAI: That's a fascinating topic! Integrating Large Language Models with external knowledge can greatly enhance their capabilities and make them more useful in various applications. By incorporating external knowledge sources, such as databases, knowledge graphs, or even the internet, language models can access a vast amount of information to provide more accurate and contextually relevant responses.\n\nThere are several approaches to integrating external knowledge with language models. One common method is to pretrain the language model on a large corpus of text that includes both general knowledge and domain-specific information. This helps the model learn to generate responses that are informed by external knowledge.\n\nAnother approach is to use knowledge graphs, which rep

In [39]:
count_tokens(
    conversation_bufw,
    "Which data source types could be used to give context to the model?"
)

Spent a total of 739 tokens


{'input': 'Which data source types could be used to give context to the model?',
 'history': 'Human: I just want to analyze the different possibilities. What can you think of?\nAI: There are several possibilities when it comes to integrating Large Language Models with external knowledge. Here are a few examples:\n\n1. Knowledge Graph Integration: As I mentioned earlier, incorporating a knowledge graph into a language model can enable it to understand and reason about relationships between entities. This can be particularly useful in tasks like question answering, where the model needs to retrieve information from a knowledge base to provide accurate responses.\n\n2. Database Integration: Language models can also be integrated with databases to access structured information. This can be helpful in scenarios where the model needs to retrieve specific data or perform complex queries. For example, a language model integrated with a customer database can provide personalized responses based

In [40]:
count_tokens(
    conversation_bufw,
    "What is my aim again?"
)

Spent a total of 451 tokens


{'input': 'What is my aim again?',
 'history': "Human: Which data source types could be used to give context to the model?\nAI: There are various data source types that can be used to give context to a language model. Some examples include:\n\n1. Textual Data: Language models can be trained on large amounts of textual data, such as books, articles, or web pages. This helps the model learn grammar, vocabulary, and general knowledge about the world.\n\n2. Knowledge Bases: Knowledge bases, such as Wikipedia or Wikidata, can provide structured information about entities, their attributes, and relationships. Integrating a knowledge base with a language model allows it to access factual information and reason about it.\n\n3. Databases: Structured databases can be used to provide specific information or perform complex queries. For example, a customer database can give context about individual customers, their preferences, or purchase history.\n\n4. Internet Data: Language models can be conne

As we can see, it effectively 'fogot' what we talked about in the first interaction. Let's see what it 'remembers'. Given that we set k to be `1`, we would expect it remembers only the last interaction.

We need to access a special method here since, in this memory type, the buffer is first passed through this method to be sent later to the llm.

In [42]:
bufw_history = conversation_bufw.memory.load_memory_variables(
    inputs=[]
)['history']

In [ ]:
print(bufw_history)

Human: What is my aim again?
AI:  Your aim is to use data sources to give context to the model.


Makes sense.

On the plus side, we are shortening our conversation length when compared to buffer memory _without_ a window:

In [43]:
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}\n'
    f'Buffer window memory conversation length: {len(tokenizer.encode(bufw_history))}'
)

Buffer memory conversation length: 1009
Summary memory conversation length: 133
Buffer window memory conversation length: 50


_Practical Note: We are using `k=2` here for illustrative purposes, in most real world applications you would need a higher value for k._

### More memory types!

Given that we understand memory already, we will present a few more memory types here and hopefully a brief description will be enough to understand their underlying functionality.

#### ConversationSummaryBufferMemory

**Key feature:** _the conversation summary memory keeps a summary of the earliest pieces of conversation while retaining a raw recollection of the latest interactions._

#### ConversationKnowledgeGraphMemory

This is a super cool memory type that was introduced just [recently](https://twitter.com/LangChainAI/status/1625158388824043522). It is based on the concept of a _knowledge graph_ which recognizes different entities and connects them in pairs with a predicate resulting in (subject, predicate, object) triplets. This enables us to compress a lot of information into highly significant snippets that can be fed into the model as context. If you want to understand this memory type in more depth you can check out [this](https://apex974.com/articles/explore-langchain-support-for-knowledge-graph) blogpost.

**Key feature:** _the conversation knowledge graph memory keeps a knowledge graph of all the entities that have been mentioned in the interactions together with their semantic relationships._

In [ ]:
# you may need to install this library
# !pip install -qU networkx

In [44]:
conversation_kg = ConversationChain(
    llm=llm,
    memory=ConversationKGMemory(llm=llm)
)

In [45]:
count_tokens(
    conversation_kg,
    "My name is human and I like mangoes!"
)

Spent a total of 1228 tokens


{'input': 'My name is human and I like mangoes!',
 'history': '',
 'response': "Hello, Human! It's nice to meet you. I'm an AI and I'm here to chat with you. Mangoes are delicious! They are tropical fruits that belong to the Anacardiaceae family. They are native to South Asia but are now grown in many parts of the world. Mangoes are known for their sweet and juicy flesh, which can range in color from yellow to orange. They are also a good source of vitamins A and C, as well as dietary fiber. Do you have a favorite variety of mango?"}

The memory keeps a knowledge graph of everything it learned so far.

In [46]:
conversation_kg.memory.kg.get_triples()

[('Human', 'human', 'name is'), ('Human', 'mangoes', 'likes')]

#### ConversationEntityMemory

**Key feature:** _the conversation entity memory keeps a recollection of the main entities that have been mentioned, together with their specific attributes._

The way this works is quite similar to the `ConversationKnowledgeGraphMemory`, you can refer to the [docs](https://python.langchain.com/en/latest/modules/memory/types/entity_summary_memory.html) if you want to see it in action.

## What else can we do with memory?

There are several cool things we can do with memory in langchain. We can:
* implement our own custom memory module
* use multiple memory modules in the same chain
* combine agents with memory and other tools

If this piques your interest, we suggest you to go take a look at the memory [how-to](https://langchain.readthedocs.io/en/latest/modules/memory/how_to_guides.html) section in the docs!